## Steps:
1. In the utils.py - Instantiate db with 
```python
    db_name="postgres"
```
2. Initiliaze the LLM model.
3. Create a first chain, which will take query and return SQL query using database table schema.
4. Create a second chain which will take the SQL query as input from first chain, execute the query and provide the answer in natural language.

In [1]:
# langchain
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# local utils
from utils import db,get_schema, custom_str_parser, handle_error_query

## Load LLM and define chains

In [2]:
# Add the LLM downloaded from Ollama
ollama_llm = "gemma:7b"
llm = ChatOllama(model=ollama_llm)

In [3]:
# ------------------------------------------------------------------------------------------------------------------------

# first chain
sql_chain_template = """Based on the table schema below, write a SQL query that would answer the user's question:
{schema}

Question: {question}
SQL Query:"""

sql_chain_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Given an input question, convert it to a SQL query. ",
        ),  # Brief context
        (
            "system",
            "Respond with only the SQL query, nothing else.",
        ),  # Emphasize desired output
        ("human", sql_chain_template),
    ]
)

sql_chain = (
    RunnablePassthrough.assign(schema=get_schema)
    | sql_chain_prompt
    | llm.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
    | RunnableLambda(custom_str_parser)
)

# ------------------------------------------------------------------------------------------------------------------------

# second_chain
template = """Based on the table schema below, question, sql query, and sql response, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""  # noqa: E501

prompt_response = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Given an input question and SQL response, convert it to a natural "
            "language answer. No pre-amble.",
        ),
        ("human", template),
    ]
)

sql_run_chain = (
    RunnablePassthrough.assign(query=sql_chain)
    | RunnablePassthrough.assign(
        schema=get_schema, response=lambda x: handle_error_query(x)
    )
    | prompt_response
    | llm
    | StrOutputParser()
)

## Run Queries

#### Test if langchain can access the database

In [4]:
print(db.get_table_info())


CREATE TABLE bitcoin (
	index BIGINT, 
	trade_date TEXT, 
	volume DOUBLE PRECISION, 
	price_usd DOUBLE PRECISION, 
	price_btc DOUBLE PRECISION, 
	market_cap DOUBLE PRECISION, 
	capitalization_change_1_day DOUBLE PRECISION, 
	"USD_price_change_1_day" DOUBLE PRECISION, 
	"BTC_price_change_1_day" DOUBLE PRECISION, 
	crypto_name TEXT, 
	crypto_type DOUBLE PRECISION, 
	ticker TEXT, 
	max_supply DOUBLE PRECISION, 
	site_url TEXT, 
	github_url TEXT, 
	minable DOUBLE PRECISION, 
	platform_name TEXT, 
	industry_name TEXT
)


In [5]:
db.run("""SELECT * FROM bitcoin LIMIT 5""")

"[(0, '2016-01-01', 36278900.0, 434.33, 1.0, 6529299589.0, 0.0, 0.0, 0.0, 'Bitcoin', 0.0, 'BTC', 21000000.0, 'https://bitcoin.org/', 'https://github.com/bitcoin/', 1.0, 'XRP', 'Proof of Work (PoW)'), (1, '2016-01-02', 30096600.0, 433.44, 1.0, 6517390487.0, -0.0018239478580617, -0.0020491331476066, 0.0, 'Bitcoin', 0.0, 'BTC', 21000000.0, 'https://bitcoin.org/', 'https://github.com/bitcoin/', 1.0, 'XRP', 'Proof of Work (PoW)'), (2, '2016-01-03', 39633800.0, 430.01, 1.0, 6467429942.0, -0.0076657283462844, -0.0079134366925065, 0.0, 'Bitcoin', 0.0, 'BTC', 21000000.0, 'https://bitcoin.org/', 'https://github.com/bitcoin/', 1.0, 'XRP', 'Proof of Work (PoW)'), (3, '2016-01-04', 38477500.0, 433.09, 1.0, 6515713340.0, 0.0074656236608678, 0.0071626241250203, 0.0, 'Bitcoin', 0.0, 'BTC', 21000000.0, 'https://bitcoin.org/', 'https://github.com/bitcoin/', 1.0, 'XRP', 'Proof of Work (PoW)'), (4, '2016-01-05', 34522600.0, 431.96, 1.0, 6500393256.0, -0.0023512519966079, -0.0026091574499527, 0.0, 'Bitcoin

#### We can run the first chain independently to test 1. We are getting valid SQL queries 2. The format of response is correct i.e. just the query

In [6]:
sql_chain.invoke({"question": "Give me list of platform names"})

'SELECT platform_name\nFROM bitcoin\nGROUP BY platform_name'

In [35]:
sql_chain.invoke({"question": "Give me list of industry names from the table."})

'SELECT industry_name\nFROM bitcoin\nGROUP BY industry_name'

In [57]:
sql_chain.invoke({"question": "What was the biggest 1 day capitalization change for Bitcoin crypto?"})

"SELECT MAX(capitalization_change_1_day) AS biggest_1_day_cap_change\nFROM bitcoin\nWHERE crypto_name = 'Bitcoin'\nGROUP BY trade_date"

In [58]:
sql_chain.invoke({"question": "Give me top five crypto names by market_cap"})

'SELECT crypto_name\nFROM bitcoin\nGROUP BY crypto_name\nORDER BY market_cap\nLIMIT 5;'

In [76]:
sql_chain.invoke({"question": "Give me number of trades for Media industry"})

"SELECT COUNT(*) AS num_trades\nFROM bitcoin\nWHERE industry_name = 'Media'"

In [79]:
sql_chain.invoke({"question": "Give me number of trades for Binance Chain platform"})

"SELECT COUNT(*) AS num_trades\nFROM bitcoin\nWHERE platform_name = 'Binance Chain'"

### Invoking the main second chain on same queries as above.

In [23]:
print(sql_run_chain.invoke({"question": "Give me list of platform names"}))

Sure, here is the natural language answer:

The platform names listed in the "bitcoin" table are:

Ardor, Binance Chain, Binance Coin, BitShares, Chiliz, EOS, Fusion, GXChain, Icon, INT Chain, Klaytn, Komodo, NEM, NuBits, NXT, Ontology, PIVX, Polkadot [IOU], Qtum, RSK Smart Bitcoin, TomoChain, Tron, Ubiq, undefined, VeChain, Vechain [Token], VITE, V Systems, Wanchain, Waves, XRP, Zilliqa


In [7]:
print(sql_run_chain.invoke({"question": "Give me list of industry names from the table."}))

Sure, here is the natural language answer:

The industry names from the table are:

- Abandoned
- Advertising
- AI
- Art
- Business Service
- Cannabis
- Communication
- Computing
- Crowdfunding
- eCommerce
- Education
- Energy
- Entertainment
- Exchange
- Gambling
- Gaming
- Healthcare
- Identity
- IoT
- Marketplace
- Media
- Prediction
- Privacy
- Proof of Stake (PoS)
- Proof of Work (PoW)
- Real Estate
- Social
- Software
- Stablecoin
- Storage
- Transportation
- Travel


In [74]:
print(sql_run_chain.invoke({"question": "What was the biggest 1 day capitalization change for Bitcoin crypto?"}))

Sure, here is the natural language answer:

The biggest 1-day capitalization change for Bitcoin crypto was 0.25261403326156.


In [78]:
print(sql_run_chain.invoke({"question": "Give me number of trades for Media industry"}))

Sure, here is the natural language answer:

The SQL query returned a result of 653 trades for the Media industry.


In [80]:
print(sql_run_chain.invoke({"question": "Give me number of trades for Binance Chain platform"}))

Sure, here is the natural language answer:

The provided SQL query returns the number of trades for the Binance Chain platform. The query selects the count of all rows in the `bitcoin` table where the `platform_name` column is equal to 'Binance Chain'. The result of the query is a single row with a single column called `num_trades` containing the number of trades for the platform, which is 1291.
